In [1]:
import re
import pickle
import os
from tqdm import tqdm

In [2]:
p = []
for root, _, files in os.walk("files/C2"):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        if "sejm/posiedzenia/pp" in file_path and 'ann_morphosyntax.xml' in file_path:
            p.append(file_path)

In [3]:
def lemmaswords(folder):
    all = []
    total_files = len(p)
    with tqdm(total=total_files) as pbar:
        for fff in folder:
            if os.path.isfile(fff):
                with open(fff, 'r') as f1:
                    word = ''
                    actualp = False
                    preceding = False
                    parenthesis = False
                    for line in f1:
                        if '<seg' in line:
                            seg = re.search(r'segm_(u-[0-9]*.[0-9]*)', line)
                        if '<f name="orth">' in line:
                            actualp = True
                        elif actualp:
                            word = re.search(r'<string>([A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ()]*)<\/string>', line)
                            actualp = False
                        if '<f name="base">' in line:
                            preceding = True
                        elif preceding:
                            lemma = re.search(r'<string>([A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ()]*)<\/string>', line)
                            if lemma != None:
                                if lemma.group(1) == '(':
                                    parenthesis = True
                                elif lemma.group(1) == ')':
                                    parenthesis = False
                                elif parenthesis == False:
                                     if word != None and seg != None:
                                        all.append((lemma.group(1), word.group(1), seg.group(1), fff))
                            preceding = False
            pbar.update(1)
    return all

In [4]:
with open('C2_lwsf_list.pickle', 'wb') as file:
    pickle.dump(lemmaswords(p), file)

100%|██████████| 2130/2130 [41:10<00:00,  1.16s/it] 
